#
### **Парсер: подготовка данных**

In [ ]:
from pandas import read_csv 
from data.labels import labels


path = 'data/'
label1 = labels['label1']

In [ ]:
train = read_csv(path + 'train.csv', index_col='id')
train.head()

In [ ]:
tar = read_csv(path + 'target.csv', index_col='id')
tar.head()

#
#### **nerpa extractor**

In [ ]:
import nerpa
from rules.rules import(
    Contract, Guarantee,
    Amount,
    Addition
)

exone = nerpa.Extractor(
    [Contract, Amount, Addition])
extwo = nerpa.Extractor(
    [Guarantee, Amount, Addition])

def switch_extractor(label):
    return exone if label == label1 else extwo

In [ ]:
for id, _ in train.iloc[61:64].iterrows():

    lb = _['label']
    txt = _['text']

    ex_ = switch_extractor(lb)
             
    nerpack = nerpa.NERpack(entities=ex_(txt))
    
    nerpack.show(txt)
    print(nerpack.entities) 

    print(nerpack.markstr() + '\n')    #   show mark's tags 

#
#### **false negative/positive errors**

Находим пустые записи в `extracted_part`

In [ ]:
len = tar['stop'] - tar['start']

answ_on = tar[len != 0].index
answ_off = tar[len == 0].index

Определяем ошибки 1 и 2 рода, где парсер нашел пункт, хотя ответа нет и где не нашел, но ответ есть

В идеале оба массива ошибок должны быть пустыми, но в приоритете избавиться от ошибок типа *false negative* (II), так как мы сразу отбросим эти записи и больше не сможем с ними работать

Ошибки типа *false positive* (I) могут отсеяться на следующих этапах парсинга

In [ ]:
one_check = nerpa.Extractor([Contract])
two_check = nerpa.Extractor([Guarantee])

In [ ]:
label_off = []  # search all labeloff docs

for id, _ in train.iterrows():

    lb =  _['label']
    txt = _['text']

    extractor = (one_check 
                 if lb == label1 
                 else two_check)

    if extractor.empty(txt):
        label_off.append(id) 

In [ ]:
label_on = [x for x in train.index 
            if x not in label_off]

In [ ]:
errorsI = [x for x in label_on if x in answ_off]
errorsII = [x for x in label_off if x in answ_on]

print(errorsI, errorsII)

Корректируем правила для [labels](rules/entities/label.py)

```python
CONTRACT = rule(
    or_(
        normalized('обеспечение'),
        normalized('исполнение'),
    ).repeatable(),
    normalized('настоящий').optional(),
    or_(
        normalized('контракт'),
        normalized('договор'),
    )
)
```

In [ ]:
#      >>>execute 4th, 5th code cells

Убираем из датасета записи, где парсер не нашел нужный лэйбл

In [ ]:
train = train.loc[label_on]
train.shape

#
#### **nerpa extractor**

`Extractor nerpa` сделан на [yargy](https://github.com/natasha/yargy), подсветка реализована с помощью [ipymarkup](https://github.com/natasha/ipymarkup).
Все инструменты из open source проекта `natasha`  

In [ ]:
amount_check = nerpa.Extractor([Amount])
amount_off = []

for id, _ in train.iterrows():
    txt = _['text']     
    if amount_check.empty(txt):
        amount_off.append(id)

In [ ]:
amount_off == errorsI

In [ ]:
train= train.drop(train.loc[amount_off].index)
train.shape

In [ ]:
import re
def find_entries(markstr, patterns):
    entries = []
    for __ in patterns:
        entries += [_.start() for _ in re.finditer(__, markstr)]
    return entries

In [ ]:
addonep = ['312', '132']    #   ['ADD{1,2}₽', '{1,2}ADD₽']
onep = ['12']   #   '{1,2}₽'

In [ ]:
pattern_entries = []

for id, _ in train.iterrows():

    lb = _['label']
    txt = _['text']
             
    extractor = switch_extractor(lb)
    nerpack = nerpa.NERpack()
    nerpack.add_marks(extractor(txt))
    
    markstr = (nerpack.markstr()
                    .replace('2', '1')
                    .replace('₽', '2') 
                    .replace('ADD', '3')
                    )
    pattern_entry = find_entries(markstr, addonep)

    pattern_entries.append(
        pattern_entry if pattern_entry 
        else find_entries(markstr, onep)
    ) 

In [ ]:
from operator import length_hint
length_hint(pattern_entries)

In [ ]:
# for id, _ in train.iterrows():

#     lb = _['label']
#     txt = _['text']

#     extractor = switch_extractor(lb)
             
#     nerpack = nerpa.NERpack()
#     nerpack.add_marks(extractor(txt))
    
#     nerpack.show(txt)
#     print(nerpack.entities) 

#     print(nerpack.markstr() + '\n')    #   show mark's tags 

In [ ]:
# from natasha import (
#     Segmenter,
#     MorphVocab,

#     NewsEmbedding,
#     NewsMorphTagger,
#     NewsSyntaxParser,
#     MoneyExtractor,
#     Doc

# )

# from razdel import tokenize
# from razdel import sentenize

# tokens = list(tokenize(train[0]['text']))
# for _ in tokens:
#     print(i)


# segmenter = Segmenter()
# morph_vocab = MorphVocab()

# emb = NewsEmbedding()
# morph_tagger = NewsMorphTagger(emb)
# syntax_parser = NewsSyntaxParser(emb)


# text = text[spans[0].start:spans[-1].stop]
# text = 'Способ обеспечения исполнения контракта, гарантийных обязательств'
# doc = Doc(text)

# doc.segment(segmenter)
# doc.tag_morph(morph_tagger)
# doc.parse_syntax(syntax_parser)

# sent = doc.sents[0]
# # sent.morph.print()


# sent.syntax.print()